In [4]:
#AI Lost Media's Text to Video Colab Workspace https://youtube.com/@ailostmedia
#huge thanks to Camenduru https://twitter.com/camenduru and Cerspense https://twitter.com/cerspense for putting these models together.
#tutorial: https://www.ailostmedia.com/post/the-ai-lost-media-text-to-video-colab-workspace
%cd /content
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!pip install git+https://github.com/huggingface/diffusers transformers accelerate imageio[ffmpeg] -U einops omegaconf decord xformers==0.0.16 safetensors
!git clone -b dev https://github.com/camenduru/Text-To-Video-Finetuning
!git clone https://github.com/ailostmedia/Potat1ALM
!mv /content/Potat1ALM/inference.py /content/Text-To-Video-Finetuning/


/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 509.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 25.5 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-msn4wjak
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-msn4wjak
  Resolved https://github.com/huggingface/diffusers to commit 2bfa55f4ed39f9dd6fed82720d87f6d4e60f21e3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 30.7 MB/s 

Cloning into 'Text-To-Video-Finetuning'...
remote: Enumerating objects: 1374, done.
remote: Counting objects: 100% (723/723), done.
remote: Compressing objects: 100% (322/322), done.
remote: Total 1374 (delta 477), reused 517 (delta 378), pack-reused 651
Receiving objects: 100% (1374/1374), 1.90 MiB | 8.79 MiB/s, done.
Resolving deltas: 100% (798/798), done.
Cloning into 'Potat1ALM'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 11 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), 5.07 KiB | 5.07 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [2]:
#@title STEP2: Install Potat1
#default 1024 x 576 - try 800 x 448 for colab
%cd /content/
!git clone https://huggingface.co/camenduru/potat1

/content
Cloning into 'potat1'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 88 (delta 14), reused 12 (delta 12), pack-reused 73
Unpacking objects: 100% (88/88), 522.56 KiB | 7.36 MiB/s, done.
Filtering content: 100% (3/3), 4.05 GiB | 58.41 MiB/s, done.


In [ ]:
%cd /content/Text-To-Video-Finetuning
import torch
import random
import numpy as np
"""
torch.use_deterministic_algorithms(True)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic=True
random.seed(2)
np.random.seed(2)
torch.manual_seed(2)
torch.cuda.manual_seed(2)
torch.cuda.manual_seed_all(2)
torch.manual_seed(0)
"""
#print("seed is " + str(torch.seed()))

#seeding = "Random"
#thisSeed = 123;


#preset = "Manual"
# while True:
#@markdown #### Be sure you have installed the model you want in step 2
model = "potat1" #@param ["potat1", "zeroscope_v2_dark_30x448x256", "zeroscope_v2_576w", "zeroscope_v2_XL"]
prompt = "BUS, door, moving, real life, HD" #@param {type:"string"}
negative = "dark, text, watermark, copyright, blurry, nsfw, noise, quick motion, bad quality, flicker, dirty, ugly, fast motion, quick cuts, fast editing, cuts" #@param {type:"string"}
prompt = f"\"{prompt}\""
negative = f"\"{negative}\""
num_steps = 25 #@param {type:"raw"}
guidance_scale = 23 #@param {type:"raw"}
width = 800 #@param {type:"raw"}
height = 448 #@param {type:"raw"}
fps = 10 #@param {type:"raw"}
num_frames = 30 #@param {type:"raw"}
seedManual = "Random"
seeding = "Random" #@param ["Random", "Manual"]

inputSeed = 7106521602475165645 #@param {type:"raw"}
if seeding == "Random":
  thisSeed = random.randint(0, ((1<<63)-1))
  print("seed is " + str(thisSeed))
else:
  thisSeed = inputSeed

thisHeight = int(round(height/8.0)*8.0)
thisWidth = int(round(width/8.0)*8.0)

thisModel="/content/"+model
!python inference.py -m {thisModel} -p {prompt} -n {negative} -W {thisWidth} -H {thisHeight} -o /content/outputs -d cuda -x -s {num_steps} -g {guidance_scale} -f {fps} -T {num_frames} -seed {thisSeed}
#-seed {thisSeed}

In [7]:
%cd /content/Text-To-Video-Finetuning
import torch
import random
import numpy as np
"""
torch.use_deterministic_algorithms(True)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic=True
random.seed(2)
np.random.seed(2)
torch.manual_seed(2)
torch.cuda.manual_seed(2)
torch.cuda.manual_seed_all(2)
torch.manual_seed(0)
"""
#print("seed is " + str(torch.seed()))

#seeding = "Random"
#thisSeed = 123;


#preset = "Manual"
# while True:
#@markdown #### Be sure you have installed the model you want in step 2
model = "potat1" #@param ["potat1", "zeroscope_v2_dark_30x448x256", "zeroscope_v2_576w", "zeroscope_v2_XL"]
prompt = "train, moving, real life, HD" #@param {type:"string"}
negative = "dark, text, watermark, copyright, blurry, nsfw, noise, quick motion, bad quality, flicker, dirty, ugly, fast motion, quick cuts, fast editing, cuts" #@param {type:"string"}
prompt = f"\"{prompt}\""
negative = f"\"{negative}\""
num_steps = 25 #@param {type:"raw"}
guidance_scale = 23 #@param {type:"raw"}
width = 800 #@param {type:"raw"}
height = 448 #@param {type:"raw"}
fps = 10 #@param {type:"raw"}
num_frames = 30 #@param {type:"raw"}
seedManual = "Random"
seeding = "Random" #@param ["Random", "Manual"]

inputSeed = 7106521602475165645 #@param {type:"raw"}
if seeding == "Random":
  thisSeed = random.randint(0, ((1<<63)-1))
  print("seed is " + str(thisSeed))
else:
  thisSeed = inputSeed

thisHeight = int(round(height/8.0)*8.0)
thisWidth = int(round(width/8.0)*8.0)

thisModel="/content/"+model
!python inference.py -m {thisModel} -p {prompt} -n {negative} -W {thisWidth} -H {thisHeight} -o /content/outputs -d cuda -x -s {num_steps} -g {guidance_scale} -f {fps} -T {num_frames} -seed {thisSeed}
#-seed {thisSeed}

/content/Text-To-Video-Finetuning
seed is 1398796689843762855
Loading pipeline components...:   0% 0/5 [00:00<?, ?it/s]2023-10-13 22:57:36.590947: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading pipeline components...: 100% 5/5 [00:06<00:00,  1.37s/it]
100% 25/25 [04:06<00:00,  9.88s/it]


In [ ]:
%cd /content/Text-To-Video-Finetuning
import torch
import random
import numpy as np
"""
torch.use_deterministic_algorithms(True)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic=True
random.seed(2)
np.random.seed(2)
torch.manual_seed(2)
torch.cuda.manual_seed(2)
torch.cuda.manual_seed_all(2)
torch.manual_seed(0)
"""
#print("seed is " + str(torch.seed()))

#seeding = "Random"
#thisSeed = 123;


#preset = "Manual"
# while True:
#@markdown #### Be sure you have installed the model you want in step 2
model = "potat1" #@param ["potat1", "zeroscope_v2_dark_30x448x256", "zeroscope_v2_576w", "zeroscope_v2_XL"]
prompt = "subway  door, door opening, moving, real life, HD" #@param {type:"string"}
negative = "dark, text, watermark, copyright, blurry, nsfw, noise, quick motion, bad quality, flicker, dirty, ugly, fast motion, quick cuts, fast editing, cuts" #@param {type:"string"}
prompt = f"\"{prompt}\""
negative = f"\"{negative}\""
num_steps = 25 #@param {type:"raw"}
guidance_scale = 23 #@param {type:"raw"}
width = 800 #@param {type:"raw"}
height = 448 #@param {type:"raw"}
fps = 10 #@param {type:"raw"}
num_frames = 30 #@param {type:"raw"}
seedManual = "Random"
seeding = "Random" #@param ["Random", "Manual"]

inputSeed = 7106521602475165645 #@param {type:"raw"}
if seeding == "Random":
  thisSeed = random.randint(0, ((1<<63)-1))
  print("seed is " + str(thisSeed))
else:
  thisSeed = inputSeed

thisHeight = int(round(height/8.0)*8.0)
thisWidth = int(round(width/8.0)*8.0)

thisModel="/content/"+model
!python inference.py -m {thisModel} -p {prompt} -n {negative} -W {thisWidth} -H {thisHeight} -o /content/outputs -d cuda -x -s {num_steps} -g {guidance_scale} -f {fps} -T {num_frames} -seed {thisSeed}
#-seed {thisSeed}

In [9]:
%cd /content/Text-To-Video-Finetuning
import torch
import random
import numpy as np
"""
torch.use_deterministic_algorithms(True)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic=True
random.seed(2)
np.random.seed(2)
torch.manual_seed(2)
torch.cuda.manual_seed(2)
torch.cuda.manual_seed_all(2)
torch.manual_seed(0)
"""
#print("seed is " + str(torch.seed()))

#seeding = "Random"
#thisSeed = 123;


#preset = "Manual"
# while True:
#@markdown #### Be sure you have installed the model you want in step 2
model = "potat1" #@param ["potat1", "zeroscope_v2_dark_30x448x256", "zeroscope_v2_576w", "zeroscope_v2_XL"]
prompt = "Announcement, HD,  megaphone, public" #@param {type:"string"}
negative = "dark, text, watermark, copyright, blurry, nsfw, noise, quick motion, bad quality, flicker, dirty, ugly, fast motion, quick cuts, fast editing, cuts" #@param {type:"string"}
prompt = f"\"{prompt}\""
negative = f"\"{negative}\""
num_steps = 25 #@param {type:"raw"}
guidance_scale = 23 #@param {type:"raw"}
width = 800 #@param {type:"raw"}
height = 448 #@param {type:"raw"}
fps = 10 #@param {type:"raw"}
num_frames = 30 #@param {type:"raw"}
seedManual = "Random"
seeding = "Random" #@param ["Random", "Manual"]

inputSeed = 7106521602475165645 #@param {type:"raw"}
if seeding == "Random":
  thisSeed = random.randint(0, ((1<<63)-1))
  print("seed is " + str(thisSeed))
else:
  thisSeed = inputSeed

thisHeight = int(round(height/8.0)*8.0)
thisWidth = int(round(width/8.0)*8.0)

thisModel="/content/"+model
!python inference.py -m {thisModel} -p {prompt} -n {negative} -W {thisWidth} -H {thisHeight} -o /content/outputs -d cuda -x -s {num_steps} -g {guidance_scale} -f {fps} -T {num_frames} -seed {thisSeed}
#-seed {thisSeed}

/content/Text-To-Video-Finetuning
seed is 4494000706456837908
Loading pipeline components...:   0% 0/5 [00:00<?, ?it/s]2023-10-13 23:12:33.349035: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading pipeline components...: 100% 5/5 [00:06<00:00,  1.39s/it]
100% 25/25 [04:03<00:00,  9.73s/it]
